# P2 - API Precio Vehículos Usados

Elaborado por GRUPO 1:
- Juanita Piraban Barbosa - 201216313
- Lorena Morales Rodríguez - 202027957
- Alejandro Barinas Guio - 201628859
- Jaime Humberto Trujillo Perea - 201920366
- Alexander Zapata Galindo - 201425426

In [ ]:
# Importar Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import sys
import plotly as pt
import plotly.express as px
import statsmodels.api as sm
#conda install py-xgboost
import xgboost as xgb

from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor

from matplotlib import pyplot
from pandas import DatetimeIndex
from pandas import Series
from tabulate import tabulate
from pandas import DataFrame

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set()  

In [ ]:
# Leer el archivo 'datos.csv'
data = pd.read_csv("DataSet/dataTraincarListings.csv")

## Introduction

- 1.2 Million listings scraped from TrueCar.com - Price, Mileage, Make, Model dataset from Kaggle
- Each observation represents the price of an used car

In [ ]:
data.head()

In [ ]:
data['Price'].describe()

In [ ]:
data.plot(kind='scatter', y='Price', x='Year')

In [ ]:
data['Mileage'].describe()

In [ ]:
data.plot(kind='scatter', y='Price', x='Mileage')

In [ ]:
data['State'].unique() 

In [ ]:
data.plot(kind='scatter', y='Price', x='State')

In [ ]:
data['Make'].unique() 

In [ ]:
data.plot(kind='scatter', y='Price', x='Make')

In [ ]:
data['Model'].unique() 

In [ ]:
data.plot(kind='scatter', y='Price', x='Model')

## Exercise P2.1 (50%)

Develop a machine learning model that predicts the price of the of car using as an input ['Year', 'Mileage', 'State', 'Make', 'Model']

#### Evaluation:
- 25% - Performance of the models using a manually implemented K-Fold (K=10) cross-validation
- 25% - Notebook explaining the process for selecting the best model. You must specify how the calibration of each of the parameters is done and how these change the performance of the model. It is expected that a clear comparison will be made of all implemented models.. Present the most relevant conslusions about the whole process.

#### Desarrollo:
Para predecir el precio de los vehículos usados, a continuación se entrenan los siguientes modelos:
- Linear Regression
- Bagging - Linear Regression
- Bagging - Decision Tree
- Random Forest
- XGBoost
- Ensemble (Decision Tree, Random Forest, XGBoost)
- Stacking (Decision Tree, Random Forest, XGBoost)

In [ ]:
# Data
X = data[['Year', 'Mileage','State','Make','Model']]
y = data['Price']

# Dummies
dum = pd.get_dummies(data[['State','Make','Model']],drop_first=True)
X_dum = pd.concat([X, dum], axis=1)
X_dum.drop(['State','Make','Model'], inplace=True, axis=1)

# Train/test dataset
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.20,
                                                    random_state=123,
                                                    shuffle=True)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

### 0. Regresión Lineal 

In [ ]:
# Features selection
a = len(X_train.filter(like='Year', axis=1).columns)
b = len(X_train.filter(like='Mileage', axis=1).columns)
c = len(X_train.filter(like='State', axis=1).columns)
d = len(X_train.filter(like='Make', axis=1).columns)
e = len(X_train.filter(like='Model', axis=1).columns)

print('Features','RMSE')

# Year
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a])
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a])
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Mileage
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b])
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b])
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/Mileage',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Mileage, State
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c])
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c])
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/Mileage/State',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Mileage, State, Make
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d])
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d])
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/Mileage/State/Make',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Mileage, State, Make, Model (all)
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e])
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e])
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/Mileage/State/Make/Model',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, State
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c]).drop(columns=['Mileage'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c]).drop(columns=['Mileage'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/State',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, State, Make
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d]).drop(columns=['Mileage'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d]).drop(columns=['Mileage'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/State/Make',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, State, Make, Model
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e]).drop(columns=['Mileage'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e]).drop(columns=['Mileage'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/State/Make/Model',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Make
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d]).drop(columns=['Mileage'],axis=1)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('State')], axis=1, inplace=True)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d]).drop(columns=['Mileage'],axis=1)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('State')], axis=1, inplace=True)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/Make',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Make, Model
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e]).drop(columns=['Mileage'],axis=1)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('State')], axis=1, inplace=True)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e]).drop(columns=['Mileage'],axis=1)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('State')], axis=1, inplace=True)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/Make/Model',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Model
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e]).drop(columns=['Mileage'],axis=1)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('State')], axis=1, inplace=True)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('Make')], axis=1, inplace=True)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e]).drop(columns=['Mileage'],axis=1)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('State')], axis=1, inplace=True)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('Make')], axis=1, inplace=True)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Year/Model',np.sqrt(mean_squared_error(y_pred, y_test)))

# Mileage
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b]).drop(columns=['Year'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b]).drop(columns=['Year'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Mileage',np.sqrt(mean_squared_error(y_pred, y_test)))

# Mileage, State
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c]).drop(columns=['Year'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c]).drop(columns=['Year'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Mileage/State',np.sqrt(mean_squared_error(y_pred, y_test)))

# Mileage, State, Make
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d]).drop(columns=['Year'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d]).drop(columns=['Year'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Mileage/State/Make',np.sqrt(mean_squared_error(y_pred, y_test)))

# Mileage, State, Make, Model
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e]).drop(columns=['Year'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e]).drop(columns=['Year'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Mileage/State/Make/Model',np.sqrt(mean_squared_error(y_pred, y_test)))

# State
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c]).drop(columns=['Year','Mileage'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c]).drop(columns=['Year','Mileage'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('State',np.sqrt(mean_squared_error(y_pred, y_test)))

# State, Make
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d]).drop(columns=['Year','Mileage'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d]).drop(columns=['Year','Mileage'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('State/Make',np.sqrt(mean_squared_error(y_pred, y_test)))

# State, Make, Model
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e]).drop(columns=['Year','Mileage'],axis=1)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e]).drop(columns=['Year','Mileage'],axis=1)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('State/Make/Model',np.sqrt(mean_squared_error(y_pred, y_test)))

# Make
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d]).drop(columns=['Year','Mileage'],axis=1)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('State')], axis=1, inplace=True)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d]).drop(columns=['Year','Mileage'],axis=1)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('State')], axis=1, inplace=True)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Make',np.sqrt(mean_squared_error(y_pred, y_test)))

# Year, Make, Model
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e]).drop(columns=['Year','Mileage'],axis=1)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('State')], axis=1, inplace=True)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e]).drop(columns=['Year','Mileage'],axis=1)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('State')], axis=1, inplace=True)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Make/Model',np.sqrt(mean_squared_error(y_pred, y_test)))

# Model
X_train_1 = pd.DataFrame(X_train.iloc[:,0:a+b+c+d+e]).drop(columns=['Year','Mileage'],axis=1)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('State')], axis=1, inplace=True)
X_train_1.drop(X_train_1.columns[X_train_1.columns.str.contains('Make')], axis=1, inplace=True)
X_test_1 = pd.DataFrame(X_test.iloc[:,0:a+b+c+d+e]).drop(columns=['Year','Mileage'],axis=1)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('State')], axis=1, inplace=True)
X_test_1.drop(X_test_1.columns[X_test_1.columns.str.contains('Make')], axis=1, inplace=True)
m = LinearRegression()
m.fit(X_train_1, y_train)
y_pred = m.predict(X_test_1)
print('Model',np.sqrt(mean_squared_error(y_pred, y_test)))

In [ ]:
# Final Model - all features
lr = LinearRegression()    
lr.fit(X_train, y_train)

# RMSE
y_pred = lr.predict(X_test)
mse0 = np.sqrt(mean_squared_error(y_pred, y_test))
print('mse',mse0)

### 1. Bagging - Linear Regression

In [ ]:
# Importance feautures

# define the model
model = DecisionTreeRegressor()

# fit the model
model.fit(X_train, y_train)

# get importance
importance = model.feature_importances_

# summarize feature importance
results=[]
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
    results.append((i,v))

# plot feature importance
#pyplot.bar([x for x in range(len(importance))], importance)
#pyplot.show()

In [ ]:
# Features = >70% of importance
X_train1=X_train.loc[:,['Year','Mileage','Model_Silverado','Make_GMC','Model_Super','Make_Land','Make_Lexus','Model_F-1504WD','Make_BMW','Make_Mercedes-Benz','Model_Wrangler','Model_Tahoe4WD','Model_Suburban4WD','Model_Escalade','Make_Kia','Make_Ram','Model_Suburban2WD','Model_Tundra','Model_Tahoe2WD','Model_TahoeLT','Model_TerrainFWD','Model_Sierra','Model_FusionSE','Model_CorvetteCoupe','Make_Ford','Model_Grand','Model_FocusSE']]
X_test1=X_test.loc[:,['Year','Mileage','Model_Silverado','Make_GMC','Model_Super','Make_Land','Make_Lexus','Model_F-1504WD','Make_BMW','Make_Mercedes-Benz','Model_Wrangler','Model_Tahoe4WD','Model_Suburban4WD','Model_Escalade','Make_Kia','Make_Ram','Model_Suburban2WD','Model_Tundra','Model_Tahoe2WD','Model_TahoeLT','Model_TerrainFWD','Model_Sierra','Model_FusionSE','Model_CorvetteCoupe','Make_Ford','Model_Grand','Model_FocusSE']]

# Features = >80% of importance
#X_train1=X_train.loc[:,['Year','Mileage','Model_Silverado','Make_GMC','Model_Super','Make_Land','Make_Lexus','Model_F-1504WD','Make_BMW','Make_Mercedes-Benz','Model_Wrangler','Model_Tahoe4WD','Model_Suburban4WD','Model_Escalade','Make_Kia','Make_Ram','Model_Suburban2WD','Model_Tundra','Model_Tahoe2WD','Model_TahoeLT','Model_TerrainFWD','Model_Sierra','Model_FusionSE','Model_CorvetteCoupe','Make_Ford','Model_Grand','Model_FocusSE','Make_Porsche','Make_Dodge','Model_25004WD','Make_Cadillac','Make_Volkswagen','Model_F-150Lariat','Model_F-250Lariat','Model_CorollaLE','Model_CruzeSedan','Make_Lincoln','Model_EscapeSE','Make_Nissan','Model_F-150SuperCrew','Model_TerrainAWD','State_ TX','Make_Tesla','Make_Jeep','Make_Hyundai','Model_CayenneAWD','State_ FL','Model_Tacoma4WD','State_ CA','Model_3','Model_Land','Model_F-350Lariat','Model_4Runner4WD','Model_MalibuLT','Model_TundraSR5','Model_MDXAWD','Model_Yukon']]
#X_test1=X_test.loc[:,['Year','Mileage','Model_Silverado','Make_GMC','Model_Super','Make_Land','Make_Lexus','Model_F-1504WD','Make_BMW','Make_Mercedes-Benz','Model_Wrangler','Model_Tahoe4WD','Model_Suburban4WD','Model_Escalade','Make_Kia','Make_Ram','Model_Suburban2WD','Model_Tundra','Model_Tahoe2WD','Model_TahoeLT','Model_TerrainFWD','Model_Sierra','Model_FusionSE','Model_CorvetteCoupe','Make_Ford','Model_Grand','Model_FocusSE','Make_Porsche','Make_Dodge','Model_25004WD','Make_Cadillac','Make_Volkswagen','Model_F-150Lariat','Model_F-250Lariat','Model_CorollaLE','Model_CruzeSedan','Make_Lincoln','Model_EscapeSE','Make_Nissan','Model_F-150SuperCrew','Model_TerrainAWD','State_ TX','Make_Tesla','Make_Jeep','Make_Hyundai','Model_CayenneAWD','State_ FL','Model_Tacoma4WD','State_ CA','Model_3','Model_Land','Model_F-350Lariat','Model_4Runner4WD','Model_MalibuLT','Model_TundraSR5','Model_MDXAWD','Model_Yukon']]

In [ ]:
# set a seed for reproducibility
seed = np.random.seed(123)
n_samples = X_train1.shape[0]
n_B = 100

# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

# grow each tree deep
baglr = LinearRegression()

# DataFrame for storing predicted from each tree
y_pred_all = pd.DataFrame(index=X_test1.index, columns=[list(range(n_B))])

# grow one tree for each bootstrap sample and make predictions on testing data
for i in range(n_B):
    X_train_1 = X_train1.iloc[samples[i], :]
    y_train_1 = y_train.iloc[samples[i]]
    baglr.fit(X_train_1, y_train_1)
    y_pred_all[i] = baglr.predict(X_test1)
    
# mse
mse1 = np.sqrt(mean_squared_error(y_pred_all.mean(axis=1), y_test))
print('mse',mse1)

### 2. Bagging - Decision Tree

In [ ]:
# set a seed for reproducibility
seed = np.random.seed(123)
n_samples = X_train1.shape[0]
n_B = 100

# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

# grow each tree deep
bagtree = DecisionTreeRegressor(max_depth=None, random_state=seed)

# DataFrame for storing predicted from each tree
y_pred_all = []

# grow one tree for each bootstrap sample and make predictions on testing data
for i in range(n_B):
    X_train_1 = X_train1.iloc[samples[i], :]
    y_train_1 = y_train.iloc[samples[i]]
    bagtree.fit(X_train_1, y_train_1)
    y_pred_all.append(bagtree.predict(X_test1))
    
# mse
y_pred = np.transpose(pd.DataFrame(y_pred_all))
mse2 = np.sqrt(mean_squared_error(y_pred.mean(axis=1), y_test))
print('mse',mse2)

### 3. Random Forest

In [ ]:
# set a seed for reproducibility
seed = np.random.seed(123)
max_fig = X_train1.shape[1]
n_B = 100

# model
clf = RandomForestRegressor(max_depth=None, n_estimators=n_B, max_features=max_fig, random_state=seed, n_jobs=-1)
clf.fit(X_train1, y_train)
y_pred = clf.predict(X_test1)

# mse
mse3 = np.sqrt(mean_squared_error(y_pred, y_test))
print('mse',mse3)

### 4. XGBoost

### 5. Ensemble (Decision Tree, Random Forest, XGBoost)

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    models['rf'] = RandomForestRegressor()
    models['dt'] = DecisionTreeRegressor()
    models['xgb'] = XGBRegressor()
    return models

## get the models to evaluate
models = get_models()

# train all the models
for model in models.keys():
    models[model].fit(X_train1, y_train)

# predict test for each model
y_pred = pd.DataFrame(index=X_test1.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test1)

# evaluate each model
print("mse results")
for model in models.keys():
    print(model,np.sqrt(mean_squared_error(y_pred[model], y_test)))

# evaluate the mean of the predictions
mse5 = np.sqrt(mean_squared_error(y_pred.mean(axis=1), y_test))
print("ens ",mse5)

### 6. Stacking (Decision Tree, Random Forest, XGBoost)

In [ ]:
# define the base models
level0 = list()
level0.append(('rf', RandomForestRegressor()))
level0.append(('df', DecisionTreeRegressor()))
level0.append(('xgb', XGBRegressor()))

# define meta learner model
level1 = LinearRegression()

# define the stacking ensemble
stk = StackingRegressor(estimators=level0, final_estimator=level1, cv=10)

# fit the model on all available data
stk.fit(X_train1, y_train)

# make a prediction 
y_pred = stk.predict(X_test1)

# mse
mse6 = np.sqrt(mean_squared_error(y_pred, y_test))
print("mse",mse6)

### Compración de modelos

In [ ]:
# MSE
table_MSE = [['Linear Regression', round(mse0,2)],['Bagging(Linear Regression)', round(mse1,2)],
            ['Bagging(Decision Tree)', round(mse2,2)],['Random Forest', round(mse3,2)],
             ['XGBoost', round(0,2)], ['Ensemble', round(mse5,2)],
             ['Stacking', round(mse6,2)]]
headers = ['Modelo', 'MSE ']
print(tabulate(table_MSE,headers,stralign="decimal"))

### Modelo Final Seleccionado

In [464]:
final_model = LinearRegression()    
final_model.fit(X_dum, y)

LinearRegression()

In [463]:
# input 
year=2019
mileage=85000
state='TX'
make='BMW'
model='3'

# list
list_all = X_dum.columns

# parameters
forecast = pd.DataFrame(np.zeros((1,613)),columns=list_all)  
forecast['Year'] = year
forecast['Mileage'] = mileage    
state_ = 'State_ ' + state
forecast['%s'%state_] = 1 
make_ = 'Make_' + make
forecast['%s'%make_] = 1    
model_ = 'Model_' + model
forecast['%s'%model_] = 1

# forecast
print('Estimated Price of Car')
print('year:',year)
print('mileage:',mileage)
print('state:',state)
print('make:',make)
print('model:',model)
print('forecast price:',final_model.predict(forecast))

Estimated Price of Car
year: 2019
mileage: 85000
state: TX
make: BMW
model: 3
forecast price: [29340.59826589]


### Conclusiones

# Exercise P2.2 (50%)
Create an API of the model.

#### Evaluation:
- 40% - API hosted on a cloud service
- 10% - Show screenshots of the model doing the predictions on the local machine

#### Desarrollo:
Para desarrollar la API se siguieron los siguientes pasos:
- Compilar modelo final (archivo *.pkl)
- Probar modelo en batch
- Crear API
- Ejecutar API
- Probar API

### Compilar modelo final (archivo *pkl)

In [465]:
import pickle

In [466]:
pickle.dump(final_model, open('API/phishing_clf.pkl', 'wb'))

### Probar modelo en batch

In [ ]:
from API.m09_model_deployment import predict_price

In [ ]:
# predict_price(year,mileage,state,make,model)
predict_price(2019,85000,'TX','BMW','3')

### Crear API

In [ ]:
#pip install Flask

In [ ]:
from flask import Flask, request
from API.m09_model_deployment import predict_price

In [ ]:
app = Flask(__name__)

@app.route('/predict', methods=['GET'])
def URLpredict():
    return {
         "Price of Car": predict_price(request.args.get('URL'))
        }, 200

### Ejecutar API

In [ ]:
app.run(debug=True, use_reloader=False, host='0.0.0.0')

### Check API - Local Host

http://localhost:5000/predict?URL=2019

### Check API - Amazon Web Service (AWS)

1. Iniciar sesión en AWS
2. Inicializar/Reiniciar la instancia
3. Conectar a la instancia
4. Ejecutar la API
5. Validar funcionamiento en link: http://ec2-18-222-147-200.us-east-2.compute.amazonaws.com:8888/predict?URL=2021